## Pregunta 3 – Extensión 2 (2,5 puntos).
Supón que activar un centro de almacenamiento (A, B o C) para transferir datos conlleva un coste
fijo adicional de 5000 €. Ajusta las variables, las restricciones y la función objetivo en el modelo
de la pregunta 2 para captar este supuesto.

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# --- 1. DATOS DEL PROBLEMA ---

# Conjuntos de Orígenes (i) y Destinos (j)
Origenes = ['Lisboa', 'Madrid', 'Turin']
Destinos = ['Paris', 'Berlin', 'Varsovia']
Rutas = gp.tuplelist([(i, j) for i in Origenes for j in Destinos])

# Oferta/Capacidad máxima de cada origen (S_i, en MilGb)
# Nota: Ahora son capacidades máximas, no flujos obligatorios.
Oferta = {
    'Lisboa': 5,
    'Madrid': 6,
    'Turin': 7
}

# Demanda de cada destino (D_j, en MilGb)
Demanda = {
    'Paris': 4,
    'Berlin': 5,
    'Varsovia': 9
}

# Costos Unitarios de Transferencia (en €/MilGb)
Costo_Unitario_Centimos = {
    ('Lisboa', 'Paris'): 4, ('Lisboa', 'Berlin'): 3, ('Lisboa', 'Varsovia'): 6,
    ('Madrid', 'Paris'): 7, ('Madrid', 'Berlin'): 4, ('Madrid', 'Varsovia'): 9,
    ('Turin', 'Paris'): 9, ('Turin', 'Berlin'): 5, ('Turin', 'Varsovia'): 2
}
Costo_Unitario = {k: v / 100 for k, v in Costo_Unitario_Centimos.items()}

# Parámetros de Costos Fijos y Capacidad
COSTO_FIJO_ORIGEN = 5000.0  # Costo por activar un centro de almacenamiento
COSTO_FIJO_CANAL_BASE = 50.0
CAPACIDAD_CANAL = 10.0
MAX_CANALES_BASE = 4

# Parámetros del Canal Adicional
COSTO_FIJO_ADICIONAL = 65.0

In [3]:
# --- 2. CREACIÓN DEL MODELO ---

m = gp.Model("TransporteCostoFijoOrigen")

Set parameter Username
Set parameter LicenseID to value 2718415
Academic license - for non-commercial use only - expires 2026-10-06


In [4]:
# --- 3. VARIABLES DE DECISIÓN ---

# x[i, j]: Flujo de datos (continuas)
x = m.addVars(Rutas, vtype=GRB.CONTINUOUS, name="Flujo")

# y[i, j]: Número de canales activados entre i y j (enteras)
y = m.addVars(Rutas, vtype=GRB.INTEGER, name="NumCanales")

# z: Alquiler del canal adicional (binaria)
z = m.addVar(vtype=GRB.BINARY, name="CanalAdicional")

# a[i]: Activación del origen i (binaria, 1 si el origen se utiliza)
a = m.addVars(Origenes, vtype=GRB.BINARY, name="ActivacionOrigen")

In [5]:
# --- 4. FUNCIÓN OBJETIVO ---
# Minimizar Z''' = Costo Variable + Costo Fijo Canales + Costo Fijo Adicional + Costo Fijo Orígenes
Costo_Variable = gp.quicksum(Costo_Unitario[i, j] * x[i, j] for i, j in Rutas)
Costo_Fijo_Canales = gp.quicksum(COSTO_FIJO_CANAL_BASE * y[i, j] for i, j in Rutas)
Costo_Fijo_Adicional = COSTO_FIJO_ADICIONAL * z
Costo_Fijo_Origenes = gp.quicksum(COSTO_FIJO_ORIGEN * a[i] for i in Origenes)

m.setObjective(Costo_Variable + Costo_Fijo_Canales + Costo_Fijo_Adicional + Costo_Fijo_Origenes, GRB.MINIMIZE)

In [6]:
# --- 5. RESTRICCIONES ---

# A. Restricción de Demanda
# La demanda debe ser satisfecha por los orígenes que estén activos.
m.addConstrs((x.sum('*', j) == Demanda[j] for j in Destinos), name="Demanda")

# B. Restricción de Enlace Lógico: Flujo Saliente y Activación de Origen
# Si un origen envía flujo (suma x_ij > 0), debe estar activado (a[i] = 1).
# La Oferta[i] se usa como M (cota superior de flujo saliente).
m.addConstrs((x.sum(i, '*') <= Oferta[i] * a[i] for i in Origenes), name="Enlace_Origen_Activacion")

# C. Restricción de Oferta (Capacidad Máxima)
# El flujo saliente total de un origen no puede superar su capacidad (Oferta original).
# Esta restricción es redundante si la Oferta es la M, pero se incluye para claridad conceptual:
m.addConstrs((x.sum(i, '*') <= Oferta[i] for i in Origenes), name="Capacidad_Origen")

# D. Restricción de Enlace: Flujo <= Capacidad Total de Canales (x_ij <= 10 * y_ij)
m.addConstrs((x[i, j] <= CAPACIDAD_CANAL * y[i, j]
              for i, j in Rutas), name="CapacidadTotalRuta")

# E. Restricción de Número Máximo de Canales en la Red
m.addConstr(y.sum() <= MAX_CANALES_BASE + z, name="MaximoCanalesRed")

# F. Restricción de Exclusión Mutua (Privacidad en Berlín)
# Se limita el número total de canales que llegan a Berlín desde Lisboa y Madrid a 1.
m.addConstr(y['Lisboa', 'Berlin'] + y['Madrid', 'Berlin'] <= 1, name="ExclusionMutua_Berlin")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# --- 6. OPTIMIZAR Y RESOLVER ---
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 20 rows, 22 columns and 60 nonzeros
Model fingerprint: 0x64f39f5f
Variable types: 9 continuous, 13 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolved: 17 rows, 19 columns, 48 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 2
  Implied bound: 1
  MIR: 1
  Flow cover: 3
  Network: 2
  Relax-and-lift: 1

Explored 1 nodes (16 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 2: 15200.8 15201.1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.520075000000e+04, best bound 1.520035000000e+04, gap 0.

In [10]:
# --- 7. IMPRIMIR RESULTADOS ---
if m.status == GRB.OPTIMAL:
    print("\n" + "=" * 60)
    print("SOLUCIÓN ÓPTIMA ENCONTRADA (Costo Fijo por Origen)")
    print(f"Costo Total Mínimo: {m.objVal:.2f} €")
    print("=" * 60)

    # 1. Orígenes Activados y Costo Fijo
    origenes_act = [i for i in Origenes if a[i].x > 0.5]
    costo_fijo_origen_final = len(origenes_act) * COSTO_FIJO_ORIGEN
    print(f"\n--- ACTIVACIÓN DE ORÍGENES (5000 € c/u) ---")
    print(f"Orígenes Activados: {', '.join(origenes_act)}")
    print(f"Costo Fijo por Origen: {costo_fijo_origen_final:.2f} €")

    # 2. Flujos y Canales
    canales_totales = 0
    print(f"\n--- DETALLE DE FLUJOS Y CANALES ---")
    for i, j in Rutas:
        num_canales = int(y[i, j].x)
        if num_canales > 0:
            canales_totales += num_canales
            print(
                f"[{i} -> {j}]: Flujo = {x[i, j].x:.2f} MilGb | Canales = {num_canales} (Cap. {num_canales * CAPACIDAD_CANAL:.0f} MilGb)")

    # 3. Resumen de Costos Fijos
    costo_variable_final = sum(Costo_Unitario[i, j] * x[i, j].x for i, j in Rutas)
    costo_fijo_canales_final = canales_totales * COSTO_FIJO_CANAL_BASE
    costo_fijo_adicional_final = z.x * COSTO_FIJO_ADICIONAL

    print("\n--- RESUMEN DE COSTOS TOTALES ---")
    print(f"  Costo Variable: {costo_variable_final:.2f} €")
    print(f"  Costo Fijo Orígenes: {costo_fijo_origen_final:.2f} €")
    print(f"  Costo Fijo Canales Base: {costo_fijo_canales_final:.2f} € ({canales_totales} canales)")
    print(f"  Canal Adicional Alquilado (z): {'SÍ' if z.x > 0.5 else 'NO'} ({costo_fijo_adicional_final:.2f} €)")

    costo_total_final = costo_variable_final + costo_fijo_origen_final + costo_fijo_canales_final + costo_fijo_adicional_final
    print(f"  Total: {costo_total_final:.2f} €")

elif m.status == GRB.INFEASIBLE:
    print("El modelo no tiene solución factible.")



SOLUCIÓN ÓPTIMA ENCONTRADA (Costo Fijo por Origen)
Costo Total Mínimo: 15200.75 €

--- ACTIVACIÓN DE ORÍGENES (5000 € c/u) ---
Orígenes Activados: Lisboa, Madrid, Turin
Costo Fijo por Origen: 15000.00 €

--- DETALLE DE FLUJOS Y CANALES ---
[Lisboa -> Berlin]: Flujo = 5.00 MilGb | Canales = 1 (Cap. 10 MilGb)
[Madrid -> Paris]: Flujo = 4.00 MilGb | Canales = 1 (Cap. 10 MilGb)
[Madrid -> Varsovia]: Flujo = 2.00 MilGb | Canales = 1 (Cap. 10 MilGb)
[Turin -> Varsovia]: Flujo = 7.00 MilGb | Canales = 1 (Cap. 10 MilGb)

--- RESUMEN DE COSTOS TOTALES ---
  Costo Variable: 0.75 €
  Costo Fijo Orígenes: 15000.00 €
  Costo Fijo Canales Base: 200.00 € (4 canales)
  Canal Adicional Alquilado (z): NO (-0.00 €)
  Total: 15200.75 €
